# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile
import btrack
import os

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
# base_dir = '/mnt/DATA/macrohet/macrohet_images/'
base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [4]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [5]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [6]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane=1,#'sum_proj',
#                              set_channel=1,
#                              set_time = 66,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [ ]:
%%time
images = images.compute().compute()

# Load tracks

In [20]:
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/04/26 10:46:02 AM] Opening HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/04/26 10:46:03 AM] Loading tracks/obj_type_1
[INFO][2023/04/26 10:46:03 AM] Loading LBEP/obj_type_1
[INFO][2023/04/26 10:46:03 AM] Loading objects/obj_type_1 (31313, 5) (31313 filtered: None)
[INFO][2023/04/26 10:46:37 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/04/26 10:46:37 AM] Closing HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


### Load downscaled version

In [7]:
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/downscaled/{row,column}_downsc_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    ds_tracks = hdf.tracks
    ds_segmentation = hdf.segmentation
ds_napari_tracks, properties, graph = btrack.utils.tracks_to_napari(ds_tracks, ndim=2)

[INFO][2023/04/26 11:25:17 AM] Opening HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5...
[INFO][2023/04/26 11:25:17 AM] Loading tracks/obj_type_1
[INFO][2023/04/26 11:25:17 AM] Loading LBEP/obj_type_1
[INFO][2023/04/26 11:25:17 AM] Loading objects/obj_type_1 (26917, 5) (26917 filtered: None)
[INFO][2023/04/26 11:25:19 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/26 11:25:19 AM] Closing HDF file: /Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5


In [13]:
ds_napari_tracks_75, properties_75, graph_75 = btrack.utils.tracks_to_napari([track for track in ds_tracks if len(track) >= 74], ndim=2)

### Recolour tracks

In [10]:
ds_col_segmentation = btrack.utils.update_segmentation(ds_segmentation, ds_tracks,)

# Launch napari image viewer

In [12]:
viewer = napari.Viewer()

# viewer.add_image(images, 
#                  channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]], 
#                  visible = False
#                  )
# viewer.add_labels(segmentation, 
#                   #scale=(10, 1, 1,), 
# #                   color={1:'yellow'}
#                  )
# viewer.add_tracks(napari_tracks,
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Tracks [5,4]", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (100,1,1)
#                 )
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (10,1,1)
                )

viewer.add_labels(ds_col_segmentation, 
                  scale=(10, 1, 1,), 
#                   color={1:'yellow'}
                 )

<Labels layer 'ds_col_segmentation' at 0x1e90b3490>

In [14]:
viewer.add_tracks(ds_napari_tracks_75,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (10,1,1)
                )

<Tracks layer 'ds_napari_tracks_75' at 0x1ec3f7d10>

# Misc:

Downscale images, binarise masks, filter tracks

In [42]:
for i in [20, 40, 60, 80]:
    viewer.add_tracks(btrack.utils.tracks_to_napari([track for track in ds_tracks if len(track) < i],ndim=2)[0], 
                      name = i,
#                     properties=properties, 
#                     graph=graph, 
#                     name="Tracks [5,4]", 
                    blending="translucent",
                    visible=True,
                    scale = (10,1,1)
                )

In [24]:
import numpy as np
from skimage.morphology import square, binary_erosion, remove_small_objects
from tqdm.auto import tqdm

In [51]:
binary_masks = np.zeros(masks.shape, dtype = np.uint8)
for n, mask in tqdm(enumerate(masks), 
                 total = len(masks), 
                 desc = 'Progress through mask stack'):
#     binary_mask = np.zeros(mask.shape, dtype = np.uint8)
    binary_mask = binary_erosion(mask, square(2))
#     mask = remove_small_objects(mask, min_size=500)
#     for segment_ID in tqdm(range(1, np.max(mask)), 
#                            total = np.max(mask), 
#                            desc = 'Progress through number of segments', 
#                            leave = False):
#         segment = mask == segment_ID 
#         eroded_segment = binary_erosion(segment, square(5))
#         binary_mask += eroded_segment.astype(np.uint8)
    binary_masks[n] = binary_mask.astype(np.uint8) 

Progress through mask stack:   0%|          | 0/75 [00:00<?, ?it/s]

##### Downscale final image

In [61]:
from skimage.transform import rescale, resize, downscale_local_mean

In [60]:
final_frame = images[-1].compute().compute()

In [62]:
gfp = final_frame[0]
rfp = final_frame[1]
gfp_images_resized = list()
for t in tqdm(range(len(gfp)), 
              desc = f'Resizing GFP images in position {row, column}', leave = False):
    gfp_image_resized = rescale(gfp[t], 1200/6048, anti_aliasing=False)
    gfp_images_resized.append(gfp_image_resized)
gfp_images_resized = np.stack(gfp_images_resized, axis = 0)
rfp_images_resized = list()
for t in tqdm(range(len(rfp)),  
              desc = f'Resizing RFP images in position {row, column}', leave = False):
    rfp_image_resized = rescale(rfp[t], 1200/6048, anti_aliasing=False)
    rfp_images_resized.append(rfp_image_resized)
rfp_images_resized = np.stack(rfp_images_resized, axis = 0)
### stack together resized images for localisation
resized_images = np.stack([gfp_images_resized,rfp_images_resized], axis = -1)

Resizing GFP images in position ('5', '4'):   0%|          | 0/1 [00:00<?, ?it/s]

Resizing RFP images in position ('5', '4'):   0%|          | 0/1 [00:00<?, ?it/s]

In [64]:
resized_images.shape

(1, 1200, 1200, 2)